## Mount Colab

In [ ]:
# mount google drive
from google.colab import drive
import os

drive.mount('/content/drive/')
os.chdir('/content/drive/Shareddrives/CS260-Project/') # TODO: CHANGE TO MATCH YOUR DIRECTORY PATH

Referencing this: https://towardsdatascience.com/how-to-fine-tune-gpt-2-for-text-generation-ae2ea53bc272

And also this: https://colab.research.google.com/drive/1HzVnEQPoa8pg05PsWCD8CHMb6AnlH2Vr#scrollTo=EQAMpRRdrtRl 

## Install Dependencies

In [ ]:
!pip install transformers
!pip install datasets
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 62.7 MB/s 
     |████████████████████████████████| 7.6 MB 59.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 73.3 MB/s 
     |████████████████████████████████| 132 kB 69.2 MB/s 
     |████████████████████████████████| 127 kB 71.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 70.0 MB/s 
     |███████████████████

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            TextDataset, DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)
from datasets import load_dataset

## Visualization in W&B
Use this as the authentication: [hidden, use your own!]

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Handling the Data
GPT-2 has the following requirements:
1. The end of the sequence should have an <|endoftext|> character appended at the end.
2. The examples that I've seen that work use .txt files, where there is not an x, y pair but rather a single line of code. In other words, the format of each row should be "artist;topic_id lyrics <|endoftext|>". 

For now, I will focus on processing the csv files into this format to continue the pipeline process. ALSO: remember that we want a .txt file for each different topic in this case.

In [ ]:
import csv
# separate the train data into each respective cluster
MAX_WORD_LEN = 1024

# filename will be of the form: ../data/train/lda-train-6.csv
def get_num_topics(filename):
  num_topics = 0
  filename = filename.strip("-")
  num_topics = filename[-1][0] # first character of the last split is the topic number
  return int(num_topics)

# assumes the file name has columns: artist, topic_id, lyric
def read_and_chunk_examples(filename, data_split):
  topic_split = {}
  print(filename)
  with open(filename) as data:
    line = 1
    reader = csv.reader(data, delimiter=',')
    for row in reader:
      if line > 1:
        artist = row[0]
        curr_topic = int(row[1])
        lyrics = row[2].strip("#").replace("\n", " nnn ")
        full_example = artist + "; " + lyrics # I CHANGED THIS FOR THE ENSEMBLE METHOD
        if (len(full_example.split(" ")) > MAX_WORD_LEN):
          full_example = full_example.split(" ")[:MAX_WORD_LEN-1] # make space for the end token
          full_example = ' '.join(full_example) #+ " <|endoftext|>" # assuming this happens during training?
        if curr_topic in topic_split:
          topic_split[curr_topic].append(full_example)
        else:
          topic_split[curr_topic] = [full_example]
      line += 1
  return topic_split

In [ ]:
# assumes that the input is a dictionary, where the key is the topic id # and the value is a list of examples formatted
# properly for GPT-2
# data_type should take values = {"train", "val"}
def write_txt_chunks(topic_split, data_type):
  for topic_id in topic_split.keys():
    filename = './data/' + data_type + '/full-' + data_type + '-10topics-' + str(topic_id) + '.txt' # TODO: CHANGE THIS TO MATCH FILEPATH YOU WANT
    with open(filename, "w") as topicfile:
      lines = topic_split[topic_id] # should output a list where each element is a formatted example
      topicfile.write('\n'.join(lines))
  return

In [ ]:
# Training Data
# TODO: CHANGE THIS TO MATCH FILEPATH YOU WANT
train_filename = "./data/train/big-lda-train-10.csv"
train_topic_split = read_and_chunk_examples(train_filename, "train")
print(train_topic_split.keys())
print(train_topic_split[5][0])

# Validation Data
# TODO: CHANGE THIS TO MATCH FILEPATH YOU WANT
val_filename = "./data/val/big-lda-val-10.csv"
val_topic_split = read_and_chunk_examples(val_filename, "val")
print(val_topic_split.keys())

./data/train/big-lda-train-10.csv
dict_keys([6, 8, 0, 3, 4, 7, 1, 5, 9, 2])
orchestral manoeuvres in the dark; (she's okay) (she's okay) (hey) (she's okay) (she's okay) (hey)  goddess of love goddess of love she never slows down and she can't get enough the goddess of love the goddess of love she's holding her heart to the stars above  she can't afford dreams and her clothes are old she can't afford hope wouldn't be so bold but she's holding his heart and she won't let go again  she's a femme fatal well that's what they say but she knows he couldn't love her any other way cause she's holding him now and she won't let go again yes, she's holding him now and she won't let go again  goddess of love goddess of love she never slows down and she can't get enough the goddess of love the goddess of love she's holding her heart to the stars above  she never slows down and she doesn't play straight just cruising around in her own sweet way but she's taking her time and he's gonna get fooled agai

In [ ]:
from random import sample
for i in train_topic_split.keys():
  train_topic_split[i] = sample(train_topic_split[i], 250) # we want this many examples for the train
  val_topic_split[i] = sample(val_topic_split[i], 100)
  print(len(train_topic_split[i]))
  print(len(val_topic_split[i]))

250
100
250
100
250
100
250
100
250
100
250
100
250
100
250
100
250
100
250
100


In [ ]:
write_txt_chunks(train_topic_split, "train")
write_txt_chunks(val_topic_split, "val")

In [ ]:
def check_for_max_length(split_dict):
  for topic in split_dict.keys():
    ex_list = split_dict[topic] # list of lyrics
    for ex in ex_list:
      if len(ex.split(" ")) > MAX_WORD_LEN:
        print("Error!")
        print(len(ex.split(" ")))
        print(ex)
  return

check_for_max_length(train_topic_split)
check_for_max_length(val_topic_split)

## Train Separate Models

In [ ]:
def load_txt_dataset(train_path, test_path, tokenizer):
  #making custom torch.utils.data.Dataset class to train on
  train_dataset = load_dataset("text", data_files={"train": [train_path]}) # "./data/train/sample-100-train-0.txt"
  test_dataset = load_dataset("text", data_files={"val": [test_path]})

  print(train_dataset)
  print(test_dataset)
  #specific preparation for transformers trainer utility. mlm stands for masked 
  #language model and is false because GPT is not a masked Language Model like 
  #bert 
  data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer, mlm=False,
  )
  return train_dataset, test_dataset, data_collator

In [ ]:
print(train_topic_split.keys())

dict_keys([6, 8, 0, 3, 4, 7, 1, 5, 9, 2])


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', unk_token="\n")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
  return tokenizer(examples['text'], 
                   padding='max_length', # REMOVE THIS ???
                   truncation=True,
                   # We use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
                   # receives the `special_tokens_mask`.
                   return_special_tokens_mask=True)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# develop the datasets to be used
from collections import defaultdict



dataset_dict = defaultdict(lambda: {"train": None,
                                    "val" : None,
                                    "collator": None})
for topic in train_topic_split.keys():
  # TODO: CHANGE THIS TO MATCH FILEPATH YOU WANT
  train_path = "./data/train/full-train-10topics-" + str(topic) + ".txt"
  val_path = "./data/val/full-val-10topics-" + str(topic) + ".txt"
  train_dataset, val_dataset, data_collator = load_txt_dataset(train_path, val_path, tokenizer)
  dataset_dict[topic]["train"] = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
  dataset_dict[topic]["val"] = val_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
  dataset_dict[topic]["collator"] = data_collator

  for ex in dataset_dict[topic]["train"]["train"]["input_ids"]:
    if (len(ex) != 1024):
      print("ERROR, length is " + str(len(ex)) + "in topic " + str(topic))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-c2b243c2e0b2f11a/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-380d773b4070aaab/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 250
    })
})
DatasetDict({
    val: Dataset({
        features: ['text'],
        num_rows: 100
    })
})


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(dataset_dict[5]['train']['train'])

Dataset({
    features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 250
})


In [ ]:
for topic in train_topic_split.keys():
  wandb.init(project="model-bigfull-10topics", 
            name="topic-" + str(topic),
            entity="cs260")

  model = GPT2LMHeadModel.from_pretrained('gpt2')
  
  args = TrainingArguments(
      # TODO: CHANGE THIS TO MATCH FILEPATH YOU WANT
      output_dir='./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-' + str(topic),
      overwrite_output_dir=True,
      evaluation_strategy="epoch",
      num_train_epochs=15,
      logging_strategy="epoch", 
      save_strategy="epoch",
      save_steps=500,
      load_best_model_at_end=True,
      report_to="wandb",
      remove_unused_columns=False)
  
  trainer = Trainer(
      model=model,
      args=args,
      data_collator=dataset_dict[topic]["collator"],
      train_dataset=dataset_dict[topic]["train"]['train'],
      eval_dataset=dataset_dict[topic]["val"]['val'])
  
  trainer.train()
  # TODO: CHANGE THIS TO MATCH FILEPATH YOU WANT
  trainer.save_model("./models/gpt-2-combo/lda-big-model-10/big-data/final/topic-" + str(topic))

  wandb.finish()

wandb: Currently logged in as: rpattichis (cs260). Use `wandb login --relogin` to force relogin


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 250
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 480
  Number of trainable parameters = 124439808
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,3.376600,5.152161
2,2.968200,4.986757
3,2.858400,4.944785
4,2.732100,4.939739
5,2.654900,4.949242
6,2.567600,4.965182
7,2.433000,4.996144
8,2.442300,5.014234
9,2.334100,5.037446
10,2.307900,5.062519


***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-5/checkpoint-32
Configuration saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-5/checkpoint-32/config.json
Model weights saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-5/checkpoint-32/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-5/checkpoint-64
Configuration saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-5/checkpoint-64/config.json
Model weights saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-5/checkpoint-64/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-5/checkpo

...

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-9/checkpoint-32
Configuration saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-9/checkpoint-32/config.json
Model weights saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-9/checkpoint-32/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-9/checkpoint-64
Configuration saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-9/checkpoint-64/config.json
Model weights saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-9/checkpoint-64/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-9/checkpo

eval/loss,▄▁▁▁▂▃▄▅▆▇▇▇███
eval/runtime,▁▁▃▅▄▃▅▅███████
eval/samples_per_second,██▅▄▅▆▃▄▁▁▁▁▁▁▁
eval/steps_per_second,▇█▅▄▅▆▃▃▁▁▁▁▁▁▁
train/epoch,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇███
train/learning_rate,█▇▇▇▆▅▅▅▄▃▃▂▂▁▁
train/loss,█▆▅▄▄▃▃▃▂▂▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.25.1",
  "use_cach

Epoch,Training Loss,Validation Loss
1,3.006500,4.622090
2,2.509200,4.472970
3,2.250900,4.442589
4,2.156200,4.400561
5,1.943900,4.390493
6,1.944400,4.407686
7,1.835000,4.395021
8,1.689900,4.378864
9,1.692600,4.399937
10,1.590200,4.406155


***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-2/checkpoint-32
Configuration saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-2/checkpoint-32/config.json
Model weights saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-2/checkpoint-32/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-2/checkpoint-64
Configuration saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-2/checkpoint-64/config.json
Model weights saved in ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-2/checkpoint-64/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./models/gpt-2-combo/lda-big-model-10/big-data/checkpoints/topic-2/checkpo

# Evaluation of Each Topic Model

## Visualize some predictions
Here, we're going to randomly print a sample song generation for each topic using the validation set that we previously evaluated on. For now, we will not evaluate, jsut print to sanity check!

In [ ]:
from transformers import GenerationConfig

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

for topic in val_topic_split.keys():
  output_preds = []

  print("PRINTING OUTPUTS FOR TOPIC: " + str(topic))
  
  # TODO: CHANGE THIS TO MATCH FILEPATH YOU WANT
  filepath = "./models/gpt-2-combo/lda-big-model-10/big-data/final/topic-" + str(topic)
  model = GPT2LMHeadModel.from_pretrained(filepath)

  count = 1
  for text in val_topic_split[topic]: 
    if count <= 50:
      prompt = text.split(";")[0]

      encoded_input = tokenizer(prompt, return_special_tokens_mask=True, return_tensors='pt') #truncate=True, return_tensors='pt')#, max_length=350)
      output_lyrics = model.generate(encoded_input["input_ids"], 
                                    do_sample=True,
                                    attention_mask=encoded_input["attention_mask"],
                                    min_length=150,
                                    max_length=500,
                                    top_p=0.9,
                                    top_k=20,
                                    repetition_penalty=1.25)
      output_lyrics = tokenizer.decode(output_lyrics[0])
      output_preds.append(output_lyrics)

      # TODO: CHANGE THIS TO MATCH FILEPATH YOU WANT
      with open("./data/out/gpt-2-10-topics/big-data-10-val-output" + str(topic) + ".txt", "a") as predictions:
        predictions.write(prompt + "<START>" + output_lyrics + "<END>\n\n")
    else:
      break
    count += 1
